In [1]:
from src.models.gigagan import GigaGAN
from src.models.gigagan.text_encoder import TextEncoder

/home/kate/grad_school/GenAI/final/final-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_encoder = dict(
    dim = 64,
    depth = 120,
    vad_model_path = "/home/kate/grad_school/GenAI/final/data/private/try-2/ckpt/trained/emobank-vad-regression-7560-30.ckpt",
    vad_config_path = "/home/kate/grad_school/GenAI/final/config-inference.txt"
)

encoder = TextEncoder(**text_encoder)

gan = GigaGAN(
    generator = dict(
        style_network = dict(
            dim = 4,
            depth = 2
        ),
        image_size = 32,
        text_encoder = encoder
    ),
    discriminator = dict(
        image_size = 32,
        text_encoder = encoder
    ),
    amp = True
)

Creating text encoder


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Loading Info: {'missing_keys': ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight'], 'unexpected_keys': ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight'], 'mismatched_keys': [], 'error_msgs': []}
Loading VAD Model from: /home/kate/grad_school/GenAI/final/data/private/try-2/ckpt/trained/emobank-vad-regression-7560-30.ckpt


INFO:root:Loaded ViT-B-32 model config.


Loading Model from: /home/kate/grad_school/GenAI/final/data/private/try-2/ckpt/trained/emobank-vad-regression-7560-30.ckpt ...Finished.


/home/kate/grad_school/GenAI/final/final-env/lib/python3.12/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(
INFO:root:Loading pretrained ViT-B-32 weights (laion400m_e32).


[2025-04-27 19:02:32,389] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO:root:x86_64-linux-gnu-gcc -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -fPIC -c /tmp/tmph086_qul/test.c -o /tmp/tmph086_qul/test.o
INFO:root:x86_64-linux-gnu-gcc /tmp/tmph086_qul/test.o -laio -o /tmp/tmph086_qul/a.out




Generator: 782.11M
Discriminator: 933.39M




In [3]:
from src.models.gigagan.data import TextImageDataset

dataset = TextImageDataset(
    folder = '../datasets/laion2B-en-aesthetic',
    image_size = 32
)
dataloader = dataset.get_dataloader(batch_size = 1)
gan.set_dataloader(dataloader)

In [4]:
gan(
    steps = 10,
    grad_accum_every = 1
)

/home/kate/grad_school/GenAI/final/src/models/gigagan/gigagan_pytorch.py:469: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_images = torch.tensor(real_images.clone().detach(), device=self.device, requires_grad=True)
/home/kate/grad_school/GenAI/final/final-env/lib/python3.12/site-packages/torch/nn/functional.py:5614: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:296.)
  proj = linear(q, w, b)
/home/kate/grad_school/GenAI/final/final-env/lib/python3.12/site-packages/torch/nn/functional.py:6391: UserWarning: Memory Efficient attention on Navi31 GPU is still experimental. Enable it with TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL=1. (Triggered internally at /pytorch/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:2

G: -7.52 | MSG: 16.22 | VG: 0.00 | D: 19.50 | MSD: 6.97 | VD: 0.00 | GP: 0.00 | SSL: 0.66 | CL: 0.00 | MAL: 20.43


/home/kate/grad_school/GenAI/final/src/models/gigagan/gigagan_pytorch.py:469: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_images = torch.tensor(real_images.clone().detach(), device=self.device, requires_grad=True)
/home/kate/grad_school/GenAI/final/src/models/gigagan/gigagan_pytorch.py:469: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_images = torch.tensor(real_images.clone().detach(), device=self.device, requires_grad=True)
/home/kate/grad_school/GenAI/final/src/models/gigagan/gigagan_pytorch.py:469: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.te

OutOfMemoryError: HIP out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 15.98 GiB of which 6.00 MiB is free. Of the allocated memory 15.21 GiB is allocated by PyTorch, and 103.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_HIP_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#images = gan.generate(texts=['a sunset'], batch_size = 1) # (4, 3, 256, 256)

In [ ]:
#import matplotlib.pyplot as plt

#img = images.squeeze(0).permute(1, 2,0).numpy()

#plt.imshow(img)
#plt.show()